In [ ]:
"""
## JMP Model - global planner, any C and J
## Created Feb 7,2025
## Updated Mar 6
## Author VG

This file solves the optimal policy to maximize global welfare (at equilibrium prices, allocations, and wages) GIVEN wedges
The global planner solves one objective function: SUM_c(Y^c)/P_h
For any number of countries and sectors. 

"""



import matplotlib.pyplot as plt
import matplotlib.axes as axs
import numpy as np
import pandas as pd
import networkx as nx 
import textwrap
import seaborn as sns
import random
from scipy.optimize import fsolve
from scipy.optimize import minimize

maindir="/Users/VanyaG1/Dropbox/VANYA/PhD/PROJECTS/Sector Centrality/JMP Model/py/"

from Model_func_2_global_planner import *  
#model function: Solve price system and demand system as functions of prices. Then solve for wages using TB condition


### NB on matrix conventions
#1. First set of index are the buyer. Second set are the seller
#2. Untransformed matrices: rows correspond to first sub and super scripts - BUYER. Columns correspond to second sub/superscripts - SELLER


"""
Solve for equilibrium and optimal policy given wedges 
"""

#params 
C=3 #number of countries
J=4 #number of industries
epsilon=5 #production side elasticity of substitution
sigma=4

params=[C,J,epsilon,sigma]
L = np.arange(10, C+1 * 10, 10)

#sample values
rand=123
np.random.seed(rand)

# CES weights
alp = np.random.uniform(0, 0.1, size=(C*J,1)) # weight on intermediates overall
gam = np.random.uniform(0, 0.5, size=(C*J, C*J)) # weights on intermediates by sector: row i, col j is weight on input sector j for buying sector i
gam = gam/gam.sum(axis=1, keepdims=True) # normalize so that the rows sum to 1

#set sample wedges
chi_mat=np.random.uniform(0, 0.5, size=(C*J, C*J)) 

#set initial policy
#tau_mat_i=np.random.uniform(0, 0.5, size=(C*J, C*J)) 
#tau_L_i=np.random.uniform(0, 0.5, size=(C*J, 1))
policy_init=np.array(np.random.uniform(0, 0.5, size=((C*J)**2+C*J,1))).reshape(1,-1)

iteration=0
def GE_loop(tau_init,params,L,alp,gam,chi_mat,iteration):
    #tau_init=tau_init.reshape(-1,1)
    #reshape the policy into the matrices: tau_mat and tau_L
    tau_mat=tau_init[0:-(C*J)].reshape((C*J,C*J))
    tau_L=tau_init[-(C*J):].reshape((C*J,1))
    
    #Guess initial wage C-1 countries, c=1 wage normalized
    init_guess=2*np.ones(C-1).reshape(-1,1)

    #Run function to solve for equilibrium wages
    #model function: Solve price system and demand system as functions of prices. 
    #Then solve for wages using TB condition. Returns wage for c=2, wage for c=1 normalized to 1
    solve_eq=1
    W_solved = fsolve(solver,init_guess,args=(params,L,alp,gam,chi_mat,tau_mat,tau_L,solve_eq))

    #run solver function again at the equilibrium W to pull out allocation, prices, wages, TB
    solve_eq=0
    Y_c, Pi_c, P_h, P_vec, R_mat, R_ic, s_ij, s_L, s_h_mat, TB, TB_c0 = solver(W_solved,params,L,alp,gam,chi_mat,tau_mat,tau_L,solve_eq)
    W_out=np.concatenate((np.array([1]),W_solved), axis=0).reshape(-1,1)
    
    iteration=iteration+1
    planner_obj=-np.sum(Y_c)/P_h
    print("objective function: "+str(planner_obj))
    return planner_obj

##planner chooses optimal policy (tau-s) to maximize planner_obj
tau_solved = minimize(GE_loop,policy_init[0],args=(params,L,alp,gam,chi_mat,iteration))
#GE_loop(params,L,alp,gam,chi_mat,tau_init)

In [15]:
min(tau_solved.x)

np.float64(-0.42088544704684827)

In [16]:
tau_solved

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -68.16418942396577
        x: [ 8.340e-05  4.744e-03 ... -4.203e-01 -4.203e-01]
      nit: 261
      jac: [ 0.000e+00  9.537e-06 ...  2.861e-06 -1.907e-06]
 hess_inv: [[ 1.036e+00 -8.510e-01 ... -6.893e-04 -2.900e-03]
            [-8.510e-01  1.450e+02 ...  1.976e-01  2.183e-01]
            ...
            [-6.893e-04  1.976e-01 ...  3.332e-01  3.167e-01]
            [-2.900e-03  2.183e-01 ...  3.167e-01  4.178e-01]]
     nfev: 42547
     njev: 271

In [ ]:
###back up--
"""
Solve for equilibrium given wedges and policy
"""

#params 
C=10 #number of countries
J=20 #number of industries
epsilon=5 #production side elasticity of substitution
sigma=4

params=[C,J,epsilon,sigma]

L = np.arange(10, C+1 * 10, 10)
rand=123
#sample values
np.random.seed(rand)

# CES weights
alp = np.random.uniform(0, 0.1, size=(C*J,1)) # weight on intermediates overall
gam = np.random.uniform(0, 0.5, size=(C*J, C*J)) # weights on intermediates by sector: row i, col j is weight on input sector j for buying sector i
gam = gam/gam.sum(axis=1, keepdims=True) # normalize so that the rows sum to 1

#set sample wedges
chi_mat=np.random.uniform(0, 0.5, size=(C*J, C*J)) 

#set sample policy
tau_mat=np.random.uniform(0, 0.5, size=(C*J, C*J)) 
tau_L=np.random.uniform(0, 0.5, size=(C*J, 1))



#Guess initial wage C-1 countries, c=1 wage normalized
init_guess=2*np.ones(C-1).reshape(-1,1)
#init_guess=2*np.ones(C).reshape(-1,1)

#Run function to solve for equilibrium wages
#model function: Solve price system and demand system as functions of prices. 
#Then solve for wages using TB condition. Returns wage for c=2, wage for c=1 normalized to 1
solve_eq=1
W_solved = fsolve(solver,init_guess,args=(params,L,alp,gam,chi_mat,tau_mat,tau_L,solve_eq))

#run solver function again at the equilibrium W to pull out allocation, prices, wages, TB
solve_eq=0
Y_c, Pi_c, P_h, P_vec, R_mat, R_ic, s_ij, s_L, s_h_mat, TB, TB_c0 = solver(W_solved,params,L,alp,gam,chi_mat,tau_mat,tau_L,solve_eq)
W_out=np.concatenate((np.array([1]),W_solved), axis=0).reshape(-1,1)
Y_c

